In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.preprocessing import scale
from nltk.corpus import stopwords
import nltk

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [2]:
df = pd.read_csv('./data/announcements.csv')
print(df)

                                                  Title  \
0            Need a Ride from Indy Airport back to Rose   
1     Yet another person asking for a ride back from...   
2                           Another ride back to campus   
3                                   Ride back to Campus   
4                      Requesting a ride back to campus   
5                               Bring back your change!   
6                                        Ride from Indy   
7                         Need a Ride Form Indi Airport   
8                     Need a ride from Indy *on Monday*   
9                   Need Ride from Indy Airport to Rose   
10    Looking for a ride from Indianapolis airport t...   
11                                  Ride back from Indy   
12                                  Ride back to Campus   
13                                        Ride from IND   
14                       Ride from Indy Airport to Rose   
15                Need a ride from Indy Airport to Rose 

In [10]:
titles = df.icol(0)
body = df.icol(1)
body.shape

/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  from ipykernel import kernelapp as app


(4167,)

In [11]:
body = body.reshape(4167,1)
titles = titles.reshape(4167,1)

In [12]:
N = 4167

In [14]:
def sentence_to_words( sentence ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", sentence) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    #return( " ".join( meaningful_words ))  
    return meaningful_words

In [6]:
def sentence_to_words( sentence ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", sentence) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    #return( " ".join( meaningful_words ))  
    return meaningful_words

In [15]:
sentence_to_words(titles[0][0])

['need', 'ride', 'indy', 'airport', 'back', 'rose']

In [16]:
def email_to_words():
    list = []
    for i in range(N):
        array = sentence_to_words(titles[i][0])
        list.extend(array)
        if(isinstance(body[i][0], str)):
            array = sentence_to_words(body[i][0])
            list.extend(array)
    return list

In [17]:
list = email_to_words()
len(list)

159378

In [18]:
words = list
count = [['UNK', -1]]

In [19]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 3000

In [20]:
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = []
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [21]:
data, count, dictionary, reverse_dictionary = build_dataset(words)

In [22]:
del words  # Hint to reduce memory.

In [23]:
print('Most common words (+UNK)', count[:5])

Most common words (+UNK) [['UNK', 20378], ('rose', 3020), ('hulman', 2705), ('edu', 2386), ('please', 1651)]


In [24]:
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Sample data [17, 11, 34, 39, 42, 1, 601, 27, 63, 161] ['need', 'ride', 'indy', 'airport', 'back', 'rose', 'happy', 'new', 'year', 'everyone']


In [25]:
data_index = 0


In [26]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [27]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)

In [28]:
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],
      '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

11 ride -> 17 need
11 ride -> 34 indy
34 indy -> 11 ride
34 indy -> 39 airport
39 airport -> 34 indy
39 airport -> 42 back
42 back -> 1 rose
42 back -> 39 airport


In [29]:
# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

In [30]:
# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

In [31]:
graph = tf.Graph()

In [32]:
with graph.as_default():

    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
               tf.truncated_normal([vocabulary_size, embedding_size],
                        stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
     # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.initialize_all_variables()
    saver = tf.train.Saver()  # Gets all variables in `graph`.

In [33]:
# Step 5: Begin training.
num_steps = 100001
#num_steps = 10

In [34]:
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0


     
         # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                    print(log_str)
    final_embeddings = normalized_embeddings.eval()
    save_path = saver.save(session, "./data/model.ckpt")

Initialized
Average loss at step  0 :  199.10067749
Nearest to family: theta,
Nearest to family: theta, business,
Nearest to family: theta, business, latest,
Nearest to family: theta, business, latest, station,
Nearest to family: theta, business, latest, station, item,
Nearest to family: theta, business, latest, station, item, properties,
Nearest to family: theta, business, latest, station, item, properties, questions,
Nearest to family: theta, business, latest, station, item, properties, questions, close,
Nearest to engineering: mark,
Nearest to engineering: mark, functions,
Nearest to engineering: mark, functions, seem,
Nearest to engineering: mark, functions, seem, much,
Nearest to engineering: mark, functions, seem, much, academics,
Nearest to engineering: mark, functions, seem, much, academics, ghz,
Nearest to engineering: mark, functions, seem, much, academics, ghz, management,
Nearest to engineering: mark, functions, seem, much, academics, ghz, management, height,
Nearest to ser

In [35]:
from sklearn.manifold import TSNE
import matplotlib

import matplotlib.pyplot as plt
%pylab inline 
# Step 6: Visualize the embeddings.
def plot_with_labels(low_dim_embs, labels, filename='test.png'):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  #in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i,:]
        plt.scatter(x, y)
        plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

    plt.savefig(filename)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [36]:
tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    

len(labels)
low_dim_embs.shape

(500, 2)

In [37]:
def findembeding(word):
    if word in dictionary:
        value = dictionary[word]
        #print('in dict:%d',value)
        return final_embeddings[value]
    else:
        #print('not in dict')
        return final_embeddings[0]

In [38]:
#represent an email with a 128 dimension vector
def meanonemail(i):
    onelist = []
    result = np.ones(128)
    result = np.append([result], [result], axis=0)
    array = sentence_to_words(titles[i][0])
    onelist.extend(array)
    if(isinstance(body[i][0], str)):
        array = sentence_to_words(body[i][0])
        onelist.extend(array)
    N = len(onelist)
    
    # print(onelist)
   
    for count in range(N):
        word = onelist[count]
        embed = findembeding(word)
        
        result = np.append(result, [embed], axis=0)
    
    row = result.shape[0]
    result = result[2:row]
    
    return np.mean(result, axis=0)
        

In [41]:
result1 = meanonemail(0)
result1.shape

(128,)

In [43]:
#this result should be saved
def representeveryemail():
    result = []
    for i in range(N):
        #print(i)
        result.append(meanonemail(i))
    return result

In [44]:
result = representeveryemail()

In [45]:
import csv

In [46]:
with open('email.csv', 'w') as mycsvfile:
    thedatawriter = csv.writer(mycsvfile)
    for row in result:
        thedatawriter.writerow(row)

In [47]:
def findnearest(index):
    distancelist = []
    if(index == 0):
        for i in range(N-1):
            #print(i+1)
            dis = numpy.linalg.norm(meanonemail(0)-meanonemail(i+1)) 
            distancelist.append(dis)
        
    else:
        for i in range(index):
            #print(i)
            dis = numpy.linalg.norm(meanonemail(index)-meanonemail(i)) 
            distancelist.append(dis)
        for i in range(N-index-1):
            #print(i+index+1)
            dis = numpy.linalg.norm(meanonemail(index)-meanonemail(i+index+1)) 
            distancelist.append(dis)
    ind = np.argmin(distancelist)
    print('original email')
    print(body[index][0])
    print('nearest email found')
    print(body[ind][0])
    #return distancelist

In [48]:
findnearest(700)

original email
Selling SAW video game for the PS3, I have gotten all of the enjoyment i can out of this game and have not picked it up in a while.  The game still works and I'm asking for about 20$ rather not go any lower than that but will entertain offers.
nearest email found
I'm looking for someone to practice playing tennis with during the weekends over the summer. All levels are welcome! If interested, please contact me at caijy@rose-hulman.edu or 6462097956
